training GPT2 on text captions only

In [ ]:
!pip install transformers


In [ ]:
#loading caption text with exluding all other data from train data of 100m strict dataset

file_path = '/content/drive/MyDrive/data/text_data.zip (Unzipped Files)/train_100M/open_subtitles.train'


with open(file_path, 'r') as file:
    data = file.read()

print(data[:500])

Buffered data was truncated after reaching the output size limit.

In [ ]:
import os

num_characters = len(data)
print(f"Total number of characters in the file: {num_characters}")

# Get the file size in bytes
file_size = os.path.getsize(file_path)
print(f"File size: {file_size / (1024 * 1024):.2f} MB")

Total number of characters in the file: 106026268
File size: 101.23 MB


In [ ]:
import re

def preprocess_text(data):
    # Step 1: Remove text within parentheses
    data = re.sub(r'\([^)]*\)', '', data)

    # Remove words that are in all uppercase letters (assuming these are sound effects or similar)
    data = re.sub(r'\b[A-Z]+\b', '', data)




    # Step 4: Convert to lowercase and normalize spaces

    data = re.sub(r'\s+', ' ', data).strip()

    # Step 5: Optional - Remove punctuation
    # data = re.sub(r'[^\w\s]', '', data)

    return data

# Apply preprocessing
cleaned_data = preprocess_text(data)

# Display a sample of the cleaned data
print(cleaned_data[:500])


- Dad. - We're going right now, Martin, very far away George, why are you waiting? Let's go. He's still in here. Peter, he's still here. Get the boy out of this house. Out of this house! Dad, 'm cold. George, beg of you, let's go! 'm not finished, Peter. It's not over yet. Where's Mom and Carole? They're in a safe place. Martin? Peter. Martin. : Well, hello, George. command you, in the name of the Lord... Save your prayers to your faith. You've done nothing but hunt me down all this time. And fo


In [ ]:
from transformers import GPT2Tokenizer

# Loading the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


In [ ]:
# Tokenize the data
tokens = tokenizer.encode(cleaned_data, add_special_tokens=False)
# Printing few tokens
print(tokens[:10])

Token indices sequence length is longer than the specified maximum sequence length for this model (25953595 > 1024). Running this sequence through the model will result in indexing errors


[12, 17415, 13, 532, 775, 821, 1016, 826, 783, 11]


In [ ]:
# Get unique tokens
unique_tokens = set(tokens)
vocab_size_in_data = len(unique_tokens)

# Display the results
print(f"Vocabulary size in the tokenized data: {vocab_size_in_data}")

Vocabulary size in the tokenized data: 43957


In [ ]:
# Maximum sequence length for GPT-2
max_length = 1024

# Split the token list into chunks of max_length
token_chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]

# Display the number of chunks
print(f"Total number of chunks: {len(token_chunks)}")


Total number of chunks: 25346


In [ ]:
#prepariong the model for training
from  transformers import AutoTokenizer,GPT2LMHeadModel,AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=1024,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token=tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)
#training the model

from huggingface_hub import notebook_login

notebook_login()

KeyboardInterrupt: 